# Analysis

## 1.Set-up

In [31]:
from util.db.connector import Connector
import pandas as pd
import numpy as np
import itertools

connector = Connector(logger=None)
wnrr_filter = {
   "config.run.data.dataset": "wnrr",
   "config.run.training.message_passing.messanger_function": "distmult",
    "config.run.training.message_passing.aggregation_function": "pna",
    "config.run.data.negative_sampling.n_negative_samples": 32,
    "config.run.training.adversarial_temperature": 1,
    "config.run.training.batch_size": 128,
    "config.run.training.message_passing.query_dependent_edge_representations": False,
    "config.run.data.remove_easy_edges": False,
    "config.run.training.message_passing.edge_representation_dimensionalities": 6,
    "config.run.training.message_passing.message_augmentation": True,
    "config.run.training.message_passing.indicator_function_as_bounding": True,
    "config.run.training.message_passing.skip_connection": True,
    "config.run.training.scoring.type": "o",
    "config.run.training.message_passing.query_embedding_dimensionality": 32,
    "config.run.training.scoring.augment_with_query_embedding": True,
    "config.run.training.add_self_edges": False,
    "config.run.training.message_passing.new_representation_based_only_on_update": False
}

fbk_filter = {
   "config.run.data.dataset": "fb15k-237",
   "config.run.training.message_passing.messanger_function": "distmult",
    "config.run.training.message_passing.aggregation_function": "pna",
    "config.run.training.adversarial_temperature": 0.5,
    "config.run.data.negative_sampling.n_negative_samples": 32,
    "config.run.training.batch_size": 256,
    "config.run.training.message_passing.query_dependent_edge_representations": True,
    "config.run.data.remove_easy_edges": True,
    "config.run.training.message_passing.edge_representation_dimensionalities": 6,
    "config.run.training.message_passing.message_augmentation": True,
    "config.run.training.message_passing.indicator_function_as_bounding": True,
    "config.run.training.message_passing.skip_connection": True,
    "config.run.training.message_passing.query_embedding_dimensionality": 32,
    "config.run.training.scoring.type": "o",
    "config.run.training.scoring.augment_with_query_embedding": True,
    "config.run.training.add_self_edges": False,
    "config.run.training.message_passing.new_representation_based_only_on_update": False
}

def flatten_dict(nested_dict):
    res = {}
    if isinstance(nested_dict, dict):
        for k in nested_dict:
            flattened_dict = flatten_dict(nested_dict[k])
            for key, val in flattened_dict.items():
                key = list(key)
                key.insert(0, k)
                res[tuple(key)] = val
    else:
        res[()] = nested_dict
    return res


def nested_dict_to_df(values_dict):
    flat_dict = flatten_dict(values_dict)
    df = pd.DataFrame.from_dict(flat_dict, orient="index")
    df.index = pd.MultiIndex.from_tuples(df.index)
    df = df.unstack(level=-1)
    df.columns = df.columns.map("{0[1]}".format)
    return df

def fill_nas_with_defaults(df: pd.DataFrame):
    na_filling_defaults = {
        "config.run.training.message_passing.message_augmentation": True,
        "config.run.training.message_passing.indicator_function_as_bounding": True,
        "config.run.training.message_passing.skip_connection": True,
        "config.run.training.scoring.type": "o",
        "config.run.training.scoring.augment_with_query_embedding": True,
        "config.run.training.add_self_edges": False,
        "config.run.training.message_passing.new_representation_based_only_on_update": False,
        "config.run.training.message_passing.initial_s_dependent_on_p": True,
        "config.run.training.message_passing.learned_zero_vector": False,

    }
    for k,v in na_filling_defaults.items():
        df[k] = df[k].fillna(v)
    return df

def filter_for_experiment(df: pd.DataFrame, dataset_filter:dict, **kwargs):
    filter_ = np.ones(len(df),dtype=int)
    df = fill_nas_with_defaults(df)
    for k,v in dataset_filter.items():
        if k in kwargs:
            v = kwargs[k]
        if k == "config.run.training.message_passing.edge_representation_dimensionalities":
            condition = [len(x) == v for x in list(df["config.run.training.message_passing.edge_representation_dimensionalities"])]
        else:
            condition = df[k] == v
        filter_ = filter_ * np.array(condition)
    return df[np.array(filter_,dtype=bool)]

all_searches = connector.db.collection("Search").get()
search_ids = [x.id for x in all_searches]
runs_and_results = []
for id in search_ids:
    runs_and_results.append(
        (
            [x.to_dict() for x in connector.db.collection("Search").document(id).collection("Runs").get()]  ,
             [x.to_dict() for x in connector.db.collection("Search").document(id).collection("Test").get()]
        )
    )
runs_and_results_clean = [(pd.json_normalize(runs),pd.json_normalize(test)) for runs,test in runs_and_results if test and runs]
runs_and_results_merged = [x.merge(y, left_on="trial_number",right_on="run_id") for x,y in runs_and_results_clean ]
all_searches = pd.concat(runs_and_results_merged)
all_searches.to_csv("all_searches.csv")

In [32]:
all_searches

,trial_number,start_date,run_id_x,config.run.save_results,config.run.evaluation.filter,config.run.evaluation.ranking_scenario,config.run.evaluation.hits_at_N,config.run.training.batch_size,config.run.training.mini_batch_size,config.run.training.optimizer.other_arguments_kwargs,...,epoch,mr,mrr,run_id_y,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10,config.run.training.message_passing.new_representation_based_only_on_update,config.run.training.message_passing.initial_s_dependent_on_p,config.run.training.message_passing.learned_zero_vector
0,0,2023-02-18 12:16:09.864783+00:00,1fd4dd2f-1624-48c5-b944-20d88b2d1447,True,True,pessimistic,"[1, 3, 10]",128,32,None,...,17,638.611267,0.551404,0,0.494770,0.577967,0.659039,NaN,NaN,NaN
1,1,2023-02-18 23:17:01.300701+00:00,9d6bd05e-7682-4080-b863-3a11f7005f09,True,True,pessimistic,"[1, 3, 10]",128,32,None,...,19,635.931152,0.557175,1,0.502125,0.582543,0.666558,NaN,NaN,NaN
0,0,2023-03-01 17:15:42.415641+00:00,30d869b4-1eac-4023-aceb-e1da461358d4,True,True,pessimistic,"[1, 3, 10]",64,32,None,...,18,637.483215,0.558537,0,0.505604,0.581012,0.666667,False,NaN,NaN
0,0,2023-02-06 13:40:48.051804+00:00,723424f6-c865-4685-89dd-8fea1dba370a,True,True,pessimistic,"[1, 3, 10]",128,32,None,...,18,8382.252930,0.360100,0,0.354691,0.358450,0.368258,NaN,NaN,NaN
0,0,2023-03-04 12:40:00.774958+00:00,c96da6e0-5a28-4746-b4f2-d31d7231b98b,True,True,pessimistic,"[1, 3, 10]",128,32,None,...,16,1.336719,0.940064,0,0.895312,0.982813,0.995313,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,0,2023-03-09 20:13:54.601590+00:00,96a15361-08d0-4190-9f68-5364f1d1a7a0,True,True,pessimistic,"[1, 3, 10]",64,32,None,...,9,674.757935,0.550537,0,0.494557,0.576849,0.660743,False,True,False
0,0,2023-02-05 10:46:04.454020+00:00,f87cbaa0-6a45-4092-aaa0-9a24990e52dd,True,True,pessimistic,"[1, 3, 10]",128,32,None,...,19,628.848145,0.483130,0,0.388526,0.545440,0.656097,NaN,NaN,NaN
0,0,2023-02-08 21:28:08.653477+00:00,436d6480-d264-4b3e-96f4-bcca64c8b9aa,True,True,pessimistic,"[1, 3, 10]",256,32,None,...,13,111.601357,0.415543,0,0.322934,0.455123,0.598305,NaN,NaN,NaN
0,0,2023-03-01 21:32:08.775539+00:00,4cc81faf-565e-4192-ad4b-91a76c172225,True,True,pessimistic,"[1, 3, 10]",128,32,None,...,20,650.344360,0.547759,0,0.494279,0.567833,0.658058,True,NaN,NaN


## WNRR replication

In [33]:
wnrr_replications = filter_for_experiment(all_searches,wnrr_filter,**{})

In [34]:
wnrr_replications

,trial_number,start_date,run_id_x,config.run.save_results,config.run.evaluation.filter,config.run.evaluation.ranking_scenario,config.run.evaluation.hits_at_N,config.run.training.batch_size,config.run.training.mini_batch_size,config.run.training.optimizer.other_arguments_kwargs,...,epoch,mr,mrr,run_id_y,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10,config.run.training.message_passing.new_representation_based_only_on_update,config.run.training.message_passing.initial_s_dependent_on_p,config.run.training.message_passing.learned_zero_vector
0,8,2023-02-08 03:09:02.583584+00:00,e1c42664-b339-4d38-81fd-c8fce315d61b,True,True,pessimistic,"[1, 3, 10]",128,32,None,...,19,630.012390,0.553377,8,0.498529,0.578457,0.662798,False,True,False
2,13,2023-02-08 17:26:05.942884+00:00,2b443178-6fca-4f85-8551-baf2d1b08e59,True,True,pessimistic,"[1, 3, 10]",128,32,None,...,16,628.517639,0.555045,13,0.500654,0.576659,0.663289,False,True,False
10,11,2023-02-08 14:43:04.845086+00:00,146f0c79-fb75-4f40-a1ec-e8e2f1fac82f,True,True,pessimistic,"[1, 3, 10]",128,32,None,...,19,630.488037,0.553884,11,0.498039,0.578294,0.663779,False,True,False


In [35]:
# MRR
print(
    wnrr_replications.mrr.mean()
)
print(
    wnrr_replications.mrr.std()
)
wnrr_replications.mrr.var()
(
        wnrr_replications.mrr.mean()  - 1.96 * wnrr_replications.mrr.std(),
         wnrr_replications.mrr.mean()  + 1.96 * wnrr_replications.mrr.std()
)

0.554101844628652
0.0008549728254292972


(0.5524260978908105, 0.5557775913664934)

In [36]:
# MR
print(
    wnrr_replications.mr.mean()
)
print(
    wnrr_replications.mr.std()
)
(
        wnrr_replications.mr.mean()  - 1.96 * wnrr_replications.mr.std(),
         wnrr_replications.mr.mean()  + 1.96 * wnrr_replications.mr.std()
)

629.6726888020834
1.0281851813443805


(627.6574458466483, 631.6879317575184)

In [37]:
print(wnrr_replications["hits_at_N.hits_at_1"].mean())
print(wnrr_replications["hits_at_N.hits_at_1"].std())
(
        wnrr_replications["hits_at_N.hits_at_1"].mean()  - 1.96 * wnrr_replications["hits_at_N.hits_at_1"].std(),
         wnrr_replications["hits_at_N.hits_at_1"].mean()  + 1.96 * wnrr_replications["hits_at_N.hits_at_1"].std()
)

0.49907376368840534
0.0013901471308925314


(0.49634907531185596, 0.5017984520649547)

In [38]:
print(wnrr_replications["hits_at_N.hits_at_3"].mean())
print(wnrr_replications["hits_at_N.hits_at_3"].std())
(
        wnrr_replications["hits_at_N.hits_at_3"].mean()  - 1.96 * wnrr_replications["hits_at_N.hits_at_3"].std(),
         wnrr_replications["hits_at_N.hits_at_3"].mean()  + 1.96 * wnrr_replications["hits_at_N.hits_at_3"].std()
)

0.5778031945228577
0.0009942448763878358


(0.5758544745651375, 0.5797519144805778)

In [39]:
print(wnrr_replications["hits_at_N.hits_at_10"].mean())
print(wnrr_replications["hits_at_N.hits_at_10"].std())
(
        wnrr_replications["hits_at_N.hits_at_10"].mean()  - 1.96 * wnrr_replications["hits_at_N.hits_at_10"].std(),
         wnrr_replications["hits_at_N.hits_at_10"].mean()  + 1.96 * wnrr_replications["hits_at_N.hits_at_10"].std()
)

0.6632886528968811
0.0004903674125671387


(0.6623275327682495, 0.6642497730255127)

## FB15k replication

In [40]:
fb_replication = filter_for_experiment(all_searches,fbk_filter,**{})

In [41]:
fb_replication

,trial_number,start_date,run_id_x,config.run.save_results,config.run.evaluation.filter,config.run.evaluation.ranking_scenario,config.run.evaluation.hits_at_N,config.run.training.batch_size,config.run.training.mini_batch_size,config.run.training.optimizer.other_arguments_kwargs,...,epoch,mr,mrr,run_id_y,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10,config.run.training.message_passing.new_representation_based_only_on_update,config.run.training.message_passing.initial_s_dependent_on_p,config.run.training.message_passing.learned_zero_vector
0,0,2023-02-21 17:20:45.775984+00:00,db16e8f5-523b-487f-b703-61d1d49d7920,True,True,pessimistic,"[1, 3, 10]",256,32,None,...,13,115.415985,0.411571,0,0.317707,0.451727,0.594591,False,True,False
1,1,2023-02-23 20:29:37.824040+00:00,36e538f9-d092-4ce4-b209-f103ebb5cc9e,True,True,pessimistic,"[1, 3, 10]",256,32,None,...,12,111.471054,0.413383,1,0.319636,0.453193,0.597474,False,True,False
0,0,2023-02-08 21:28:08.653477+00:00,436d6480-d264-4b3e-96f4-bcca64c8b9aa,True,True,pessimistic,"[1, 3, 10]",256,32,None,...,13,111.601357,0.415543,0,0.322934,0.455123,0.598305,False,True,False


In [42]:
print(
    fb_replication.mrr.mean()
)
print(
    fb_replication.mrr.std()
)
(
        fb_replication["mrr"].mean()  - 1.96 * fb_replication["mrr"].std(),
         fb_replication["mrr"].mean()  + 1.96 * fb_replication["mrr"].std()
)

0.4134991963704427
0.001988650431548579


(0.4096014415246075, 0.4173969512162779)

In [43]:
# MR
print(
    fb_replication.mr.mean()
)
print(
    fb_replication.mr.std()
)
(
        fb_replication["mr"].mean()  - 1.96 * fb_replication["mr"].std(),
         fb_replication["mr"].mean()  + 1.96 * fb_replication["mr"].std()
)

112.82946523030598
2.2409391974795936


(108.43722440324598, 117.22170605736599)

In [44]:
print(fb_replication["hits_at_N.hits_at_1"].mean())
print(fb_replication["hits_at_N.hits_at_1"].std())
(
        fb_replication["hits_at_N.hits_at_1"].mean()  - 1.96 * fb_replication["hits_at_N.hits_at_1"].std(),
         fb_replication["hits_at_N.hits_at_1"].mean()  + 1.96 * fb_replication["hits_at_N.hits_at_1"].std()
)

0.3200925091902415
0.002643625956062233


(0.31491100231635955, 0.3252740160641235)

In [45]:
print(fb_replication["hits_at_N.hits_at_3"].mean())
print(fb_replication["hits_at_N.hits_at_3"].std())
(
        fb_replication["hits_at_N.hits_at_3"].mean()  - 1.96 * fb_replication["hits_at_N.hits_at_3"].std(),
         fb_replication["hits_at_N.hits_at_3"].mean()  + 1.96 * fb_replication["hits_at_N.hits_at_3"].std()
)

0.4533476730187734
0.0017031324339378472


(0.4500095334482552, 0.4566858125892916)

In [46]:
print(fb_replication["hits_at_N.hits_at_10"].mean())
print(fb_replication["hits_at_N.hits_at_10"].std())
(
        fb_replication["hits_at_N.hits_at_3"].mean()  - 1.96 * fb_replication["hits_at_N.hits_at_3"].std(),
         fb_replication["hits_at_N.hits_at_3"].mean()  + 1.96 * fb_replication["hits_at_N.hits_at_3"].std()
)

0.5967899560928345
0.0019488386766004348


(0.4500095334482552, 0.4566858125892916)

## Different Path Lengths

In [47]:
dfs = []
for i in [1,2,4,8]:
    dfs.append(filter_for_experiment(all_searches,wnrr_filter,**{
        "config.run.training.message_passing.edge_representation_dimensionalities": i
    }))

In [48]:
pd.concat(dfs)[[
    "config.run.training.message_passing.edge_representation_dimensionalities",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.training.message_passing.edge_representation_dimensionalities,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
0,[32],8382.252930,0.360100,0.354691,0.358450,0.368258
6,"[32, 32]",1753.554199,0.422301,0.401275,0.429716,0.460281
0,"[32, 32, 32, 32]",889.356323,0.537406,0.491174,0.556718,0.636646
0,"[32, 32, 32, 32, 32, 32, 32, 32]",530.184204,0.555297,0.497712,0.579928,0.669990


## Message and Aggregation Functions

In [49]:
dfs = []
for msg in ["transe","distmult","rotate"]:
    for agg in ["sum","mean","max","pna"]:
        dfs.append(filter_for_experiment(all_searches,wnrr_filter,**{
            "config.run.training.message_passing.aggregation_function": agg,
            "config.run.training.message_passing.messanger_function": msg

        }))

In [50]:
pd.concat(dfs)[[
    "config.run.training.message_passing.aggregation_function",
    "config.run.training.message_passing.messanger_function",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.training.message_passing.aggregation_function,config.run.training.message_passing.messanger_function,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
2,sum,transe,719.037109,0.451002,0.355181,0.510297,0.617522
5,sum,transe,707.038391,0.428198,0.317424,0.502452,0.619974
4,mean,transe,748.809204,0.414866,0.302223,0.488558,0.610167
1,max,transe,702.348450,0.520140,0.465021,0.543151,0.632396
0,pna,transe,673.300415,0.536406,0.482347,0.560150,0.644655
3,sum,distmult,647.255615,0.538037,0.482347,0.560314,0.650539
8,mean,distmult,692.632385,0.528277,0.474011,0.547728,0.640569
7,max,distmult,717.447021,0.534928,0.481530,0.556064,0.642530
0,pna,distmult,630.012390,0.553377,0.498529,0.578457,0.662798
2,pna,distmult,628.517639,0.555045,0.500654,0.576659,0.663289


## Batchsize

In [51]:
dfs = []
for size in [8,16,32,64,256,512]:
        dfs.append(filter_for_experiment(all_searches,wnrr_filter,**{
            "config.run.training.batch_size": size,
        }))
pd.concat(dfs)[[
    "config.run.training.batch_size",
    "config.run.training.mini_batch_size",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.training.batch_size,config.run.training.mini_batch_size,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
0,32,32,652.472168,0.551427,0.495914,0.573063,0.667211
0,64,32,637.483215,0.558537,0.505604,0.581012,0.666667
0,256,32,634.529358,0.555944,0.501876,0.577619,0.665302


## Negative Examples


In [52]:
dfs = []
for size in [16,64,128,256]:
        dfs.append(filter_for_experiment(all_searches,wnrr_filter,**{
            "config.run.data.negative_sampling.n_negative_samples": size,
        }))
pd.concat(dfs)[[
    "config.run.data.negative_sampling.n_negative_samples",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.data.negative_sampling.n_negative_samples,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
0,16,646.195801,0.546974,0.493789,0.569140,0.654135
0,64,638.236145,0.556580,0.504413,0.576659,0.665904
0,128,614.150818,0.562127,0.508663,0.586957,0.666558


## Temperature Experiments

In [53]:
dfs = []
for temp in [0,0.25,0.5,0.75,1.25,1.5]:
        dfs.append(filter_for_experiment(all_searches,wnrr_filter,**{
            "config.run.training.adversarial_temperature": temp,
        }))

In [54]:
pd.concat(dfs)[[
    "config.run.training.adversarial_temperature",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]



,config.run.training.adversarial_temperature,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
1,0.00,615.180603,0.518848,0.459464,0.541680,0.637463
2,0.25,666.123047,0.556298,0.502452,0.578947,0.663289
4,0.50,641.848633,0.556275,0.500817,0.583524,0.666558
3,0.75,634.212952,0.554682,0.499673,0.577640,0.660837
0,1.25,623.718323,0.553902,0.498039,0.580582,0.662962
6,1.50,629.704956,0.552945,0.498529,0.576496,0.661491


## 1-Hop

In [55]:
filter_for_experiment(all_searches,wnrr_filter,**{
            "config.run.data.remove_easy_edges": True,
        })[[
    "config.run.data.remove_easy_edges",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.data.remove_easy_edges,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
0,True,628.848145,0.48313,0.388526,0.54544,0.656097


## 1-Hop FBK

In [56]:
filter_for_experiment(all_searches,fbk_filter,**{
            "config.run.data.remove_easy_edges": False,
        })[[
    "config.run.data.remove_easy_edges",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.data.remove_easy_edges,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
0,False,116.265282,0.371872,0.270655,0.412298,0.575512


## Dimensionality


In [57]:
dfs = []
for dim in [16,64,128]:
        dfs.append(filter_for_experiment(all_searches,wnrr_filter,**{
             "config.run.training.message_passing.query_embedding_dimensionality": dim,
        }))
pd.concat(dfs)[[
    "config.run.training.message_passing.query_embedding_dimensionality",
    "config.run.training.mini_batch_size",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]


,config.run.training.message_passing.query_embedding_dimensionality,config.run.training.mini_batch_size,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
0,16,32,643.491943,0.541565,0.485616,0.566362,0.652010
0,64,16,641.100037,0.561086,0.507519,0.583197,0.669173
0,128,8,634.982666,0.563211,0.509153,0.585322,0.669663


## Query Dependent Edge Representations

In [58]:
filter_for_experiment(all_searches,wnrr_filter,**{
            "config.run.training.message_passing.query_dependent_edge_representations": True,
        })[[
    "config.run.training.message_passing.query_dependent_edge_representations",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.training.message_passing.query_dependent_edge_representations,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
0,True,601.982971,0.555044,0.500817,0.577313,0.665087


## FBK No query dependent edge reps

In [59]:
filter_for_experiment(all_searches,fbk_filter,**{
            "config.run.training.message_passing.query_dependent_edge_representations": False,
        })[[
    "config.run.data.remove_easy_edges",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.data.remove_easy_edges,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
0,True,117.228714,0.408476,0.313383,0.448283,0.5953


## Skip Connection

In [60]:
filter_for_experiment(all_searches,wnrr_filter,**{
               "config.run.training.message_passing.skip_connection": False,
        })[[
    "config.run.training.message_passing.skip_connection",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.training.message_passing.skip_connection,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
9,False,620.077637,0.55525,0.500163,0.577967,0.665413


## Augmented Scoring

In [61]:
filter_for_experiment(all_searches,wnrr_filter,**{
              "config.run.training.scoring.augment_with_query_embedding": False
        })[[
    "config.run.training.message_passing.skip_connection",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.training.message_passing.skip_connection,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
7,True,897.614075,0.552018,0.49951,0.573553,0.657077


## Self Edges

In [62]:
filter_for_experiment(all_searches,wnrr_filter,**{
        "config.run.training.add_self_edges": True,
        "config.run.training.message_passing.message_augmentation": False,
        "config.run.training.message_passing.new_representation_based_only_on_update": False,
        })[[
    "config.run.training.add_self_edges",
    "config.run.training.message_passing.message_augmentation",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.training.add_self_edges,config.run.training.message_passing.message_augmentation,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
8,True,False,649.546753,0.554448,0.502452,0.574371,0.659039


In [63]:
filter_for_experiment(all_searches,wnrr_filter,**{
        "config.run.training.add_self_edges": False,
        "config.run.training.message_passing.message_augmentation": True,
        "config.run.training.message_passing.new_representation_based_only_on_update": True,
        })[[
    "config.run.training.add_self_edges",
    "config.run.training.message_passing.message_augmentation",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.training.add_self_edges,config.run.training.message_passing.message_augmentation,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
3,False,True,633.340454,0.551585,0.499183,0.570775,0.659039


## SO-Scoring

In [64]:
filter_for_experiment(all_searches,wnrr_filter,**{
        "config.run.training.scoring.type": "so",
        })[[
    "config.run.training.scoring.type",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.training.scoring.type,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
5,so,532.707214,0.555308,0.499673,0.577967,0.664269


## O-Scoring excluding P

In [65]:
filter_for_experiment(all_searches,wnrr_filter,**{
        "config.run.training.scoring.type": "o",
        "config.run.training.scoring.augment_with_query_embedding": False,
        })[[
    "config.run.training.scoring.type",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.training.scoring.type,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
7,o,897.614075,0.552018,0.49951,0.573553,0.657077


## Scoring Functions

In [66]:
dfs = []
for type in ["transe", "rotate", "distmult"]:
    for type2 in ["transe", "rotate", "distmult"]:

        dfs.append(filter_for_experiment(all_searches, wnrr_filter, **{
           "config.run.training.scoring.type": type,
           "config.run.training.message_passing.query_dependent_edge_representations": True,
            "config.run.training.message_passing.messanger_function": type2,
        }))

pd.concat(dfs)[[
    "config.run.training.scoring.type",
     "config.run.training.message_passing.messanger_function",
    "mr",
    "mrr",
    "hits_at_N.hits_at_1",
    "hits_at_N.hits_at_3",
    "hits_at_N.hits_at_10"
]]

,config.run.training.scoring.type,config.run.training.message_passing.messanger_function,mr,mrr,hits_at_N.hits_at_1,hits_at_N.hits_at_3,hits_at_N.hits_at_10
0,transe,transe,1338.422485,0.391882,0.372344,0.391141,0.424812
0,transe,distmult,859.743164,0.425052,0.390324,0.428898,0.493462
0,rotate,rotate,846.805603,0.417686,0.388362,0.419418,0.479405
0,rotate,distmult,781.163940,0.424751,0.390324,0.429879,0.492154
0,distmult,distmult,549.639709,0.550923,0.497221,0.574698,0.656751
